<a href="https://colab.research.google.com/github/dlsun/Stat305-S20/blob/master/colabs/notebooks/STAT_305_Notebook_10_Confidence_Intervals_in_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I encourage you to work through this notebook with a partner so that you can discuss your answers. You should meet over an application such as Discord or Zoom. One person can share their screen with this notebook open.

In [ ]:
!pip install -q symbulate
from symbulate import *

import matplotlib.pyplot as plt

# Confidence Intervals in Practice

In the previous lesson, we learned that if we have i.i.d. data $X_1, ..., X_n$ from a $\text{Normal}(\mu, \sigma)$ distribution where $\sigma$ is known, then 

$$ \bar X \pm z_{1-\alpha}\frac{\sigma}{\sqrt{n}} $$

is a confidence interval for $\mu$. The critical value $z_{1-\alpha}$ controls the confidence level. For example, if the critical value is $z_{.95} = 2$, then the interval is a 95% confidence interval. In general, a $(1 - \alpha)$ confidence interval is an interval with a $(1 - \alpha)$ probability of covering $\mu$.

The confidence intervals for $\mu$ that we constructed in the last lesson were unrealistic for two reasons:

1. We had to assume that the data $X_1, ..., X_n$ come from a normal distribution.
2. We had to know the standard deviation $\sigma$ of each observation.

While these assumptions might make sense for repeated measurements of a physical phenomenon (such as tire pressure), they are not reasonable for most data sets. What do we do if we want a confidence interval for $\mu$ but are not willing to assume normality or a value for the standard deviation?

## Central Limit Theorem to the Rescue

The Central Limit Theorem (partly) saves us. It says that no matter the distribution of $X_1, ..., X_n$, $\bar X$ is _approximately_ normal when $n$ is large. Therefore, we can pretend that $\bar X$ is normal for the purposes of constructing confidence intervals.

So the interval above
$$ \bar X \pm z_{1-\alpha} \frac{\text{SD}[X_1]}{\sqrt{n}} $$
has _approximately_ a $(1 -\alpha)$ probability of covering $\mu$. The closer that $\bar X$ is to normal, the closer the coverage probability is to $1-\alpha$.

However, this still assumes that $\text{SD}[X_1]$ is known. Often, we will also need to estimate $\text{SD}[X_1]$ from the data as well. A generic estimator of $\text{SD}[X_1]$, that works in all cases, is the sample standard deviation:

$$ S = \sqrt{\frac{1}{n-1} \sum_{i=1}^n (X_i - \bar X )^2 }. $$

In specific situations, there may be better estimators of $\text{SD}[X_1]$. (We will see an example below.)

# Example 2 Revisited

Recall Example 2, where we wanted to estimate the rate parameter $\lambda$ of a Poisson process by which radioactive particles reach a Geiger counter.

We recorded the number of particles in 1-second intervals for 10 seconds:
$$ 0, 3, 1, 0, 0, 1, 0, 2, 0, 4. $$

Recall that an unbiased estimator of $\lambda$ is the sample mean $\bar X$.

**Question 1.** Calculate a 95% confidence interval for $\lambda$ based on $\bar X$. Explain where you use the Central Limit Theorem. You will need to calculate an estimate of $\text{SD}[X_1]$. There are multiple reasonable choices; just choose one.

_YOUR ANSWER HERE_

# Coverage of Confidence Intervals

The confidence interval that we just calculated may not have a 95% probability of covering $\lambda$ because it relies on two approximations:

1. the accuracy of the normal approximation for $\bar X$
2. how close our estimate of $\text{SD}[X_1]$ is to the true value

It is useful to know how close the coverage probability is to the advertised value of 95%. We conduct a simulation study to find out.

Let's evaluate a 95% confidence interval that is constructed using the sample standard deviation $S$ (defined above), in place of the unknown value of $\text{SD}[X_1]$. Let's do the following:

- pick an arbitrary value of $\lambda$
- repeatedly simulate $n=10$ independent observations from a $\text{Poisson}(\lambda)$ distribution
- form the (approximate) 95% confidence interval from each simulated data set
- check if each interval actually contains $\lambda$

and tabulate the results.

In [ ]:
# Construct 95% confidence interval based on data
def construct_95_ci(data):
  n = len(data)
  X_bar = mean(data)
  S = sd(data)
  return X_bar - 2 * S / sqrt(n), X_bar + 2 * S / sqrt(n)

# Define lambda and a function that checks whether lambda is in an interval
lam = 0.8
def contains_lam(interval):
  lower, upper = interval
  return lower < lam < upper

# Simulate n observations from a Poisson lambda and construct 95% CI
n = 10
X = RV(Poisson(lam) ** n)
X.sim(1000).apply(construct_95_ci)

Now, how many of these intervals actually contain the true $\lambda$ of $0.8$?

In [ ]:
X.sim(1000).apply(construct_95_ci).count(contains_lam)

The 95% interval appears to cover the true $\lambda$ of $0.8$ only about 90% of the time. So the approximations seem to matter a lot.

The coverage probability might vary by $\lambda$, so we repeat the simulation for a grid of $\lambda$ values.



In [ ]:
n = 10

# Construct 95% confidence interval based on data
def construct_95_ci(data):
  n = len(data)
  X_bar = mean(data)
  S = sd(data)
  return X_bar - 2 * S / sqrt(n), X_bar + 2 * S / sqrt(n)

# Define grid of lambda values and placeholder for the coverage probabilities
lams = np.arange(start=0.2, stop=4.0, step=0.2)
coverage_probs = []

# Iterate over lambda values
for lam in lams:
  # Define a function that checks whether lam is in an interval
  def contains_lam(interval):
    lower, upper = interval
    return lower < lam < upper

  # Simulate observations from a Poisson lambda, form CI, see if it contains lambda
  X = RV(Poisson(lam) ** n)
  coverage_probs.append(
      X.sim(1000).apply(construct_95_ci).count(contains_lam) / 1000
  )

# Plot the coverage probability as a function of lambda
plt.plot(lams, coverage_probs)
plt.xlabel(r"$\lambda$")
plt.ylabel(r"Coverage Probability")
# Plot the desired coverage probability (95%)
plt.axhline(y=0.95, linestyle="--", color="red")

The confidence interval is advertised as having 95% probability of covering $\lambda$, but clearly the actual probability is quite a bit lower, closer to 90%.

This curve is quite wiggly because of simulation noise. We could make it smoother by increasing the number of simulations, but that would increase the time the simulation takes. We did 1000 simulations above for each value of $\lambda$; you may want to try increasing it to 10000 to see the effect it has on the runtime and on the output.

In the confidence interval above, we estimated $\text{SD}[X_1]$ using the sample standard deviation $S$.

However, for Poisson random variables, there is slightly different confidence interval we could construct. Because $\text{SD}[X_1] = \sqrt{\lambda}$, we could also estimate it by $\sqrt{\bar X}$, since $\bar X$ is an estimate for $\lambda$.

**Question 2.** Conduct a simulation study to analyze the coverage probability of a 95% confidence interval that uses $\sqrt{\bar X}$ as the estimator of $\text{SD}[X_1]$ in the formula, instead of the generic estimator $S$. (You can do this by modifying the simulation study above.) 

Does this confidence interval have better coverage than the confidence interval above?

In [ ]:
# YOUR CODE HERE

_YOUR ANALYSIS HERE_

# Submission Instructions

You do not need to submit this notebook. However, a solid understanding of this material is necessary to do the final project.